In [ ]:
import shnitsel as st
import shnitsel.xarray
import matplotlib.pyplot as plt

In [ ]:
frames = st.read('/nc/reports/2025-05-21_datasheets/filtered_C4H8_g0.nc')
frames

In [ ]:
# Select trajectories without cleavages
mol = frames.atXYZ.isel(frame=0).st.to_mol(to2D=False)
frames.atXYZ.attrs['smiles_map'] = st.bridges.mol_to_numbered_smiles(mol)
frames = frames.st.filter_by_length()

In [ ]:
dih = frames.atXYZ.st.dihedral(0, 1, 2, 3, full=False, deg=True)
dih

In [ ]:
from sklearn.cluster import KMeans

### $k$-means clustering using final timestep

In [ ]:
Y = dih.groupby('trajid').map(lambda traj: traj[{'frame': -1}])
kmc = KMeans(n_clusters=2)
kmc.fit(Y.data.reshape(-1, 1))
plt.scatter(Y, [1]*len(Y), c=kmc.predict(Y.data.reshape(-1, 1)))
plt.xlabel('Final dihedral / °')

### $k$-means clustering using final 2 timesteps

In [ ]:
Y = dih.groupby('trajid').map(lambda traj: traj[{'frame': slice(-2, None)}].unstack('frame').assign_coords(time=[-2,-1]))
kmc = KMeans(n_clusters=2)
kmc.fit(Y)
plt.scatter(Y[:, 0], Y[:, 1], c=kmc.predict(Y))
plt.xlabel('Penultimate dihedral / °')
plt.ylabel('Final dihedral / °')

### -10 and -1

In [ ]:
Y = dih.groupby('trajid').map(lambda traj: traj[{'frame': [-10, -1]}].unstack('frame').assign_coords(time=[-10,-1]))
kmc = KMeans(n_clusters=2)
kmc.fit(Y)
plt.scatter(Y[:, 0], Y[:, 1], c=kmc.predict(Y))
plt.xlabel('Dihedral 5 fs before end / °')
plt.ylabel('Final dihedral / °')

### $k$-means over the final 20 frames + PCA

In [ ]:
def tmp(traj):
    traj = traj[{'frame': slice(-10, None)}]
    traj = traj.unstack('frame')
    traj = traj.assign_coords(time=range(-20, 0, 2))
    return traj

Ynew = dih.groupby('trajid').map(tmp)
Ynew

In [ ]:
kmc = KMeans(n_clusters=2)
kmc.fit(Ynew)
pca_new = Ynew.st.pca('time')
plt.scatter(pca_new[:, 0], pca_new[:, 1], c=kmc.predict(Ynew))
plt.xlabel('PC1')
plt.ylabel('PC2')